# Networks

> Neural networks


In [ ]:
#| default_exp nets

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
# from IPython.display import clear_output, DisplayHandle


# def update_patch(self, obj):
#     clear_output(wait=True)
#     self.display(obj)


# DisplayHandle.update = update_patch

In [ ]:
#| hide
from fastai.vision.all import test_eq

In [ ]:
#| export
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Union

from torch import cat as torch_cat
from torch import Tensor as torch_Tensor, randn as torch_randn
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
from torch.cuda.amp import autocast
from torch import float16 as torch_float16, float32 as torch_float32

from fastai.vision.all import create_unet_model, resnet18, resnet34, resnet50, resnet101, resnet152

from mamba_ssm import Mamba

from monai.networks.blocks import Convolution
from monai.networks.layers.factories import Act, Norm, Pool
from monai.utils import set_determinism
from monai.networks.nets import BasicUNet, AttentionUnet, DynUNet, UNet, BasicUNet, ResNet, ResNetFeatures

from bioMONAI.core import get_device


In [ ]:
#| hide
device = get_device()
print(device)

cuda


## UNet

In [ ]:
#| export
def create_custom_unet(resnet_version,      # Choose a ResNet model between: 'resnet18', 'resnet34', 'resnet50', 'resnet101', and 'resnet152'.
                       n_in=1,              # Number of input channels, default is 1 (e.g., grayscale).
                       n_out=1,               # Number of output channels.
                       img_size=(128, 128), # Tuple for the input image size, default is (128, 128).
                       pretrained=True,     # If True, use a pretrained ResNet backbone.
                       cut=4,               # The cut point for the ResNet model, default is 4.
                       ):
    """
    Create a U-Net model with a ResNet backbone.

    Returns:
    - U-Net model with the specified ResNet backbone.
    """

    # Dictionary to map model names to Fastai ResNet functions
    resnet_versions = {
        'resnet18': resnet18,
        'resnet34': resnet34,
        'resnet50': resnet50,
        'resnet101': resnet101,
        'resnet152': resnet152
    }
    
    # Select the chosen ResNet model
    if resnet_version not in resnet_versions:
        raise ValueError(f"Invalid resnet_version '{resnet_version}'. Choose from: {list(resnet_versions.keys())}")
    
    resnet_fn = resnet_versions[resnet_version]

    # Create and return the U-Net model
    return create_unet_model(resnet_fn, n_out, img_size, pretrained, n_in=n_in, cut=cut)


## Denoising CNN


In [ ]:
#| export
import torch.nn as nn

class DnCNN(nn.Module):
    """
    A Deep Neural Network for Image Denoising (DnCNN) model.

    """
    def __init__(self, spatial_dims=2, # Number of spatial dimensions
                 in_channels=1, # Number of input channels
                 out_channels=1, # Number of output channels
                 num_of_layers=9, # Number of convolutional layers
                 features=64, # Number of feature maps
                 kernel_size=3, # Size of the convolution kernel
                 ):
        super(DnCNN, self).__init__()
        
        # Create a list to hold the layers of the network
        layers = []
        
        # Append the first convolution layer with specific parameters
        layers.append(Convolution(spatial_dims=spatial_dims,
                                  in_channels=in_channels,
                                  out_channels=features,
                                  kernel_size=kernel_size,
                                  bias=False,
                                  norm=None))
        
        # Append the remaining convolution layers with default parameters
        for _ in range(num_of_layers-2):
            layers.append(Convolution(spatial_dims=spatial_dims,
                                      in_channels=features,
                                      out_channels=features,
                                      kernel_size=kernel_size))
        
        # Append the final convolution layer with specific parameters
        layers.append(Convolution(spatial_dims=spatial_dims,
                                  in_channels=features,
                                  out_channels=out_channels,
                                  kernel_size=kernel_size,
                                  bias=False,
                                  norm=None))
        
        # Convert the list of layers into a sequential container
        self.dncnn = nn.Sequential(*layers)
        
    def forward(self, x, # Input image tensor with shape [batch_size, in_channels, height, width].
                ):
        """
        Forward pass of the DnCNN model.
        
        Returns:
            torch.Tensor: The denoised output image tensor with shape [batch_size, out_channels, height, width].
        """
        # Compute the residual by passing the input through the network
        residual = self.dncnn(x)
        
        # Subtract the residual from the original input to get the denoised output
        denoised = x - residual
        
        return denoised


In [ ]:
x = torch_randn(16, 1, 32, 64)

tst = DnCNN(2,1)
test_eq(tst(x).shape, x.shape)

## DeepLab v3+

`DeepLabv3` is a semantic segmentation architecture that handles the problem of segmenting objects at multiple scales. It uses the Atroys Spatial Pyramid Pooling module, and introduces various updates with respect to other versions.

### Config

The `DeeplabConfig` class has been created to centralize all settings and hyperparameters in one place. It uses two main functions: Get_padding and interpolate. Get_padding is a function that calculates the amount of padding needed for a convolution operation to get the desired output size. Interpolate is a function that allows the resizing of a tensor using interpolation.

In [ ]:
#| export
@dataclass
class DeeplabConfig:
    dimensions: int # Number of spatial dimensions
    in_channels: int # Number of input channels
    out_channels: int # Number of output channels
    backbone: str = "xception" # Backbone model for the encoder
    pretrained: bool = False # If True, use a pretrained backbone
    middle_flow_blocks: int = 16 # Number of middle flow blocks
    aspp_dilations: List[int] = field(default_factory=lambda: [1, 6, 12, 18]) # Dilation rates for the ASPP module
    entry_block3_stride: int = 2 # Stride for the third entry block
    middle_block_dilation: int = 1 # Dilation rate for the middle block
    exit_block_dilations: Tuple[int, int] = (1, 2) # Dilation rates for the exit block

def get_padding(kernel_size: int, # Size of the convolution kernel
                dilation: int, # Dilation rate
                ) -> int: # Padding size
    return (kernel_size - 1) * dilation // 2
    
def interpolate(x: torch_Tensor, # Input tensor
                size: Union[List[int], Tuple[int, ...]], # Size of the output tensor
                dims: int, # Number of spatial dimensions
                ) -> torch_Tensor: # Output tensor
    if dims == 2:
        return F.interpolate(x, size=size, mode='bilinear', align_corners=True)
    elif dims == 3:
        return F.interpolate(x, size=size, mode='trilinear', align_corners=True)
    else:
        raise ValueError(f"Unsupported number of dimensions: {dims}")

In [ ]:
show_doc(DeeplabConfig)

---

[source](https://github.com/bmandracchia/bioMONAI/blob/main/bioMONAI/nets.py#L98){target="_blank" style="float:right; font-size:smaller"}

### DeeplabConfig

>      DeeplabConfig (dimensions:int, in_channels:int, out_channels:int,
>                     backbone:str='xception', pretrained:bool=False,
>                     middle_flow_blocks:int=16,
>                     aspp_dilations:List[int]=<factory>,
>                     entry_block3_stride:int=2, middle_block_dilation:int=1,
>                     exit_block_dilations:Tuple[int,int]=(1, 2))

### Blocks

`SeparableConv` is a class that carries out a type of convolution operation that splits the traditional convolution into two parts: depthwise convolution (a convolution filter for each channel independently), and pointwise convolution (combines the outputs of the depthwise convolution).

In [ ]:
#| export
class SeparableConv(nn.Module):
    def __init__(self, config: DeeplabConfig, # Configuration for the Deeplab model
                 inplanes: int, # Number of input channels
                 planes: int, # Number of output channels
                 kernel_size: int = 3, # Size of the convolution kernel
                 stride: int = 1,  # Stride for the convolution
                 dilation: int = 1, # Dilation rate for the convolution
                 bias: bool = False, # If True, add a bias term
                 norm: Optional[str] = None, # Type of normalization layer
                 ):
        super().__init__()
        self.conv1 = Convolution(
            spatial_dims=config.dimensions, 
            in_channels=inplanes, 
            out_channels=inplanes, 
            kernel_size=kernel_size,
            groups=inplanes, 
            padding=get_padding(kernel_size, dilation), 
            dilation=dilation, 
            bias=bias, 
            strides=stride
        )
        self.pointwise = Convolution(
            spatial_dims=config.dimensions, 
            in_channels=inplanes, 
            out_channels=planes, 
            kernel_size=1, 
            strides=1,
            padding=0, 
            dilation=1, 
            groups=1, 
            bias=bias,
            norm=Norm.BATCH if norm else None
        )

    def forward(self, x: torch_Tensor) -> torch_Tensor:
        x = self.conv1(x)
        x = self.pointwise(x)
        return x

`Block` is a class that combines multiple separable convolutional layers and residual connections. 

In [ ]:

class Block(nn.Module):
    def __init__(self, config: DeeplabConfig, # Configuration for the Deeplab model
                 inplanes: int, # Number of input channels
                 planes: int, # Number of output channels
                 reps: int, # Number of convolutional layers
                 stride: int = 1,  # Stride for the convolution
                 dilation: int = 1, # Dilation rate for the convolution
                 start_with_relu: bool = True, # If True, start with a ReLU activation
                 grow_first: bool = True, # If True, increase the number of channels in the first convolution
                 is_last: bool = False, # If True, add a convolution layer at the end
                 ):
        super().__init__()
        if planes != inplanes or stride != 1:
            self.skip = Convolution(config.dimensions, inplanes, planes, kernel_size=1, bias=False, 
                                    strides=stride, norm=Norm.BATCH)
        else:
            self.skip = None

        self.relu = nn.ReLU(inplace=True)
        rep = []

        filters = inplanes
        if grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv(config, inplanes, planes, 3, stride=1, dilation=dilation, norm=Norm.BATCH))
            filters = planes

        for _ in range(reps - 1):
            rep.append(self.relu)
            rep.append(SeparableConv(config, filters, filters, 3, stride=1, dilation=dilation, norm=Norm.BATCH))

        if not grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv(config, inplanes, planes, 3, stride=1, dilation=dilation, norm=Norm.BATCH))

        if not start_with_relu:
            rep = rep[1:]

        if stride != 1:
            rep.append(SeparableConv(config, planes, planes, 3, stride=2))

        if stride == 1 and is_last:
            rep.append(SeparableConv(config, planes, planes, 3, stride=1))

        self.rep = nn.Sequential(*rep)

    def forward(self, inp: torch_Tensor) -> torch_Tensor:
        x = self.rep(inp)
        if self.skip is not None:
            skip = self.skip(inp)
        else:
            skip = inp
        x += skip
        return x

### Aligned Xception

The `Xception` class defines the Xception Neural Network used in DeepLab.

In [ ]:
#| export
class Xception(nn.Module):
    def __init__(self, config: DeeplabConfig, # Configuration for the Deeplab model
                 ):
        super().__init__()
        self.config = config

        self.conv1 = Convolution(config.dimensions, config.in_channels, 32, kernel_size=3,
                                 bias=False, strides=2, padding=1, norm=Norm.BATCH)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = Convolution(config.dimensions, 32, 64, kernel_size=3,
                                 bias=False, strides=1, padding=1, norm=Norm.BATCH)

        self.block1 = Block(config, 64, 128, reps=2, stride=2, start_with_relu=False)
        self.block2 = Block(config, 128, 256, reps=2, stride=2, start_with_relu=True, grow_first=True)
        self.block3 = Block(config, 256, 728, reps=2, stride=config.entry_block3_stride, 
                            start_with_relu=True, grow_first=True, is_last=True)

        # Middle flow
        self.middle_flow = nn.Sequential(*[
            Block(config, 728, 728, reps=3, stride=1, dilation=config.middle_block_dilation,
                  start_with_relu=True, grow_first=True)
            for _ in range(config.middle_flow_blocks)
        ])

        # Exit flow
        self.exit_block = Block(config, 728, 1024, reps=2, stride=1, dilation=config.exit_block_dilations[0],
                             start_with_relu=True, grow_first=False, is_last=True)

        self.conv3 = SeparableConv(config, 1024, 1536, 3, stride=1, dilation=config.exit_block_dilations[1], norm=Norm.BATCH)
        self.conv4 = SeparableConv(config, 1536, 1536, 3, stride=1, dilation=config.exit_block_dilations[1], norm=Norm.BATCH)
        self.conv5 = SeparableConv(config, 1536, 2048, 3, stride=1, dilation=config.exit_block_dilations[1], norm=Norm.BATCH)

    def forward(self, x: torch_Tensor) -> Tuple[torch_Tensor, torch_Tensor]:
        # Entry flow
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        x = self.block1(x)
        low_level_feat = x
        x = self.block2(x)
        x = self.block3(x)

        # Middle flow
        x = self.middle_flow(x)

        # Exit flow
        x = self.exit_block(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.conv5(x)
        x = self.relu(x)

        return x, low_level_feat

### ASPP

The class `ASPP_module` to compute the Atroys Spatial Pyramid Pooling method and create the convolution that uses it.

In [ ]:
#| export
class ASPP_module(nn.Module):
    def __init__(self, config: DeeplabConfig, # Configuration for the Deeplab model
                 inplanes: int,               # Number of input channels
                 planes: int,                 # Number of output channels
                 dilation: int,               # Dilation rate for the convolution
                 ):
        super().__init__()
        kernel_size = 1 if dilation == 1 else 3
        padding = 0 if dilation == 1 else dilation
        self.atrous_convolution = Convolution(config.dimensions, inplanes, planes, kernel_size=kernel_size,
                                              strides=1, padding=padding, dilation=dilation, 
                                              bias=False, norm=Norm.BATCH)
        self.relu = nn.ReLU()

    def forward(self, x: torch_Tensor) -> torch_Tensor:
        x = self.atrous_convolution(x)
        return self.relu(x)

### DeepLab V3

The `Deeplab` class combines the different modules to make the DeepLab V3 architecture. 

In [ ]:
#| export
class Deeplab(nn.Module):
    def __init__(self, config: DeeplabConfig):
        super().__init__()
        self.config = config

        # Choose backbone based on configuration
        if config.backbone == "xception":
            self.backbone = Xception(config)
            backbone_out_channels = 2048
            low_level_channels = 128  # This might need adjustment based on Xception implementation
        else:
            usePretrained = config.pretrained if config.in_channels==1 and config.dimensions==3 else False
            self.backbone = ResNetFeatures(config.backbone, pretrained=usePretrained, spatial_dims=config.dimensions, in_channels=config.in_channels)
            del self.backbone.fc
            del self.backbone.avgpool
            
            # Forward pass through the backbone to get the output before the final classifier
            if config.dimensions == 3:
                dummy_input = torch_randn(1, config.in_channels, 16, 128, 128) 
            elif config.dimensions == 2:
                dummy_input = torch_randn(1, config.in_channels, 128, 128) 
            dummy_output = self.backbone(dummy_input)
            
            # print("Output channels:", dummy_output[-1].shape[1])
            
            backbone_out_channels = dummy_output[-1].shape[1]
            low_level_channels = dummy_output[1].shape[1]  # first layer output
        
        # else:
        #     raise ValueError(f"Unsupported backbone: {config.backbone}")

        # ASPP
        self.aspp_modules = nn.ModuleList([
            ASPP_module(config, backbone_out_channels, 256, dilation=dilation)
            for dilation in config.aspp_dilations
        ])

        self.global_avg_pool = nn.Sequential(
            Pool[Pool.ADAPTIVEAVG, config.dimensions](1),
            Convolution(config.dimensions, backbone_out_channels, 256, kernel_size=1, strides=1, bias=False, norm=Norm.BATCH),
            nn.ReLU()
        )

        self.conv1 = Convolution(config.dimensions, 1280, 256, kernel_size=1, bias=False, norm=Norm.BATCH)
        self.conv2 = Convolution(config.dimensions, low_level_channels, 48, kernel_size=1, bias=False, norm=Norm.BATCH)

        self.last_conv = nn.Sequential(
            Convolution(config.dimensions, 304, 256, kernel_size=3, strides=1, padding=1, bias=False, norm=Norm.BATCH),
            nn.ReLU(),
            Convolution(config.dimensions, 256, 256, kernel_size=3, strides=1, padding=1, bias=False, norm=Norm.BATCH),
            nn.ReLU(),
            Convolution(config.dimensions, 256, config.out_channels, kernel_size=1, strides=1)
        )

    def forward(self, input: torch_Tensor) -> torch_Tensor:
        if self.config.backbone == "xception":
            x, low_level_features = self.backbone(input)
        else:
            x = self.backbone.conv1(input)
            x = self.backbone.bn1(x)
            x = self.backbone.act(x)
            x = self.backbone.maxpool(x)

            low_level_features = self.backbone.layer1(x)
            x = self.backbone.layer2(low_level_features)
            x = self.backbone.layer3(x)
            x = self.backbone.layer4(x)

        aspp_results = [module(x) for module in self.aspp_modules]
        x5 = self.global_avg_pool(x)
        x5 = interpolate(x5, size=x.shape[2:], dims=self.config.dimensions)
        x = torch_cat(aspp_results + [x5], dim=1)

        x = self.conv1(x)
        x = interpolate(x, size=low_level_features.shape[2:], dims=self.config.dimensions)

        low_level_features = self.conv2(low_level_features)

        x = torch_cat((x, low_level_features), dim=1)
        x = self.last_conv(x)
        x = interpolate(x, size=input.shape[2:], dims=self.config.dimensions)

        return x

#### Example

In [ ]:
# Load a pre-trained ResNet backbone
resnet_backbone = ResNetFeatures('resnet10', pretrained=False, in_channels=1, spatial_dims=3)

# Forward pass through the backbone to get the output before the final classifier
dummy_input = torch_randn(1, 1, 64, 224, 224)  # Example input size; adjust based on your needs
output = resnet_backbone(dummy_input)

# The shape of 'output' will give you the number of channels at this stage in the backbone
print("Output channels:", output[-1].shape[1])

Output channels: 512


In [ ]:
# For 2D images
config_2d = DeeplabConfig(
    dimensions=2,
    in_channels=3,  # For RGB images
    out_channels=4,
    backbone="xception",  # or whatever backbone you're using
    aspp_dilations=[1, 6, 12, 18]
)
model_2d = Deeplab(config_2d)

# For 3D images
config_3d = DeeplabConfig(
    dimensions=3,
    in_channels=1,  # For single-channel 3D medical images
    out_channels=4,
    middle_flow_blocks=16,
    aspp_dilations=[1, 6, 12, 18]
)
model_3d = Deeplab(config_3d)

In [ ]:
from torch import no_grad as torch_no_grad

In [ ]:
def test_deeplab(config, input_shape, expected_output_shape):
    set_determinism(0)  # For reproducibility
    
    model = Deeplab(config)
    model.eval()  # Set the model to evaluation mode
    
    # Generate random input tensor
    x = torch_randn(*input_shape)
    
    # Forward pass
    with torch_no_grad():
        output = model(x)
    
    # Check output shape
    assert output.shape == expected_output_shape, f"Expected shape {expected_output_shape}, but got {output.shape}"
    
    print(f"Test passed for {config.dimensions}D model with backbone {config.backbone}")
    print(f"Input shape: {input_shape}")
    print(f"Output shape: {output.shape}")
    print("---")


In [ ]:
# Test 2D model
config_2d = DeeplabConfig(
    dimensions=2,
    in_channels=3,
    out_channels=4,
    backbone="xception",
    aspp_dilations=[1, 6, 12, 18]
)
test_deeplab(config_2d, (1, 3, 64, 64), (1, 4, 64, 64))

# Test 2D model with ResNet50 backbone
config_2d_resnet = DeeplabConfig(
    dimensions=2,
    in_channels=3,
    out_channels=4,
    backbone="resnet50",
    aspp_dilations=[1, 6, 12, 18]
)
test_deeplab(config_2d_resnet, (1, 3, 64, 64), (1, 4, 64, 64))

# Test 3D model
config_3d = DeeplabConfig(
    dimensions=3,
    in_channels=1,
    out_channels=4,
    backbone="xception",
    aspp_dilations=[1, 6, 12, 18]
)
test_deeplab(config_3d, (1, 1, 64, 64, 64), (1, 4, 64, 64, 64))

# Test 3D model with ResNet10 backbone
config_3d_resnet = DeeplabConfig(
    dimensions=3,
    in_channels=1,
    out_channels=4,
    backbone="resnet10",
    aspp_dilations=[1, 6, 12, 18]
)
test_deeplab(config_3d_resnet, (1, 1, 64, 64, 64), (1, 4, 64, 64, 64))

print("All tests passed successfully!")

Test passed for 2D model with backbone xception
Input shape: (1, 3, 64, 64)
Output shape: torch.Size([1, 4, 64, 64])
---
Test passed for 2D model with backbone resnet50
Input shape: (1, 3, 64, 64)
Output shape: torch.Size([1, 4, 64, 64])
---
Test passed for 3D model with backbone xception
Input shape: (1, 1, 64, 64, 64)
Output shape: torch.Size([1, 4, 64, 64, 64])
---
Test passed for 3D model with backbone resnet10
Input shape: (1, 1, 64, 64, 64)
Output shape: torch.Size([1, 4, 64, 64, 64])
---
All tests passed successfully!


## UMamba

In [ ]:
#| export
class MambaLayer(nn.Module):
    """
    A custom neural network layer that incorporates the Mamba block from the Mamba model, 
    along with layer normalization and optional mixed precision handling.

    """
    
    def __init__(self, dim,  # Dimension of the input tensor
                 d_state=16, # Expansion factor for the state in the Mamba block
                 d_conv=4,   # Width of the local convolution in the Mamba block
                 expand=2,   # Factor by which to expand the dimensions in the Mamba block
                 ):
        super().__init__()
        self.dim = dim                      # Dimension of the input tensor
        self.norm = nn.LayerNorm(dim)       # Layer normalization
        self.mamba = Mamba(                 # Mamba block
            d_model=dim,  # Model dimension d_model
            d_state=d_state,  # SSM state expansion factor
            d_conv=d_conv,  # Local convolution width
            expand=expand  # Block expansion factor
        )
    
    @autocast(enabled=False)
    def forward(self, x, # Input tensor of shape [batch_size, dim, height, width].
                ):
        """
        Forward pass of the MambaLayer. Applies layer normalization and optionally converts input precision.
   
        Returns:
            torch.Tensor: Output tensor after applying Mamba block and normalization.
        """
        if x.dtype == torch_float16:
            x = x.type(torch_float32)  # Convert input to float32 for mixed precision handling
        
        B, C = x.shape[:2]
        assert C == self.dim  # Ensure the feature size matches the dimension of the layer
        
        n_tokens = x.shape[2:].numel()
        img_dims = x.shape[2:]
        x_flat = x.reshape(B, C, n_tokens).transpose(-1, -2)  # Flatten and transpose for Mamba input
        
        x_norm = self.norm(x_flat)  # Apply layer normalization
        
        x_mamba = self.mamba(x_norm)  # Pass through the Mamba block
        
        out = x_mamba.transpose(-1, -2).reshape(B, C, *img_dims)  # Reshape and transpose back to original dimensions
        
        return out


In [ ]:
#| export
class UMamba(DynUNet):
    """
    A custom subclass of DynUNet that integrates the Mamba layer into the model's bottleneck.
    
    This class inherits from `DynUNet` and adds a specific bottleneck structure containing a convolution block followed by a MambaLayer.
    
    """
    
    def get_bottleneck(self):
        """
        Constructs the bottleneck part of the network.
        
        The bottleneck consists of a convolution block followed by a MambaLayer. Both components are added to a sequential container.
        
        Returns:
            nn.Sequential: A PyTorch sequential container with the convolution block and MambaLayer for the bottleneck.
        """
        mamba_bottleneck = []
        # Add a convolution block before the MambaLayer in the bottleneck
        mamba_bottleneck.append(
            self.conv_block(
                self.spatial_dims,          # Spatial dimensions of the input data
                self.filters[-2],           # Number of filters for the previous layer
                self.filters[-1],           # Number of filters for this layer (output)
                self.kernel_size[-1],       # Kernel size for the convolution
                self.strides[-1],           # Stride for the convolution
                self.norm_name,             # Normalization method name
                self.act_name,              # Activation function name
                dropout=self.dropout        # Dropout probability
            )
        )
        # Add the MambaLayer to the bottleneck
        mamba_bottleneck.append(
            MambaLayer(dim = self.filters[-1])  # Initialize the MambaLayer with the current filter size as dimension
        )
        return nn.Sequential(*mamba_bottleneck)  # Return the sequential container holding both components


#### Example

In [ ]:
x = torch_randn(16, 1, 32, 64)

tst = DynUNet(2,1,1,[3,3,3],[1,1,1],[1,1])
print(tst(x).shape)
test_eq(tst(x).shape, x.shape)

torch.Size([16, 1, 32, 64])


In [ ]:
x = torch_randn(16, 1, 32, 64).cuda()

tst = UMamba(2,1,1,[3,3,3],[1,1,1],[1,1]).cuda()
print(tst(x).shape)
test_eq(tst(x).shape, x.shape)

torch.Size([16, 1, 32, 64])


---

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()